In [2]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [3]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [4]:
# 设置计算的年份
year = 2015

In [5]:
# 输入研究区的shp文件所在路径
wuhan_shp ="D:/NDVI_exercise/wuhan/wuhanregion.shp"
# shp 转 ee 
wuhan_ee = geemap.shp_to_ee(wuhan_shp)
# 添加到layer
Map.addLayer(wuhan_ee,{},"wuhan")
Map.centerObject(wuhan_ee,7)   # 设置以研究区居中

In [6]:
# 去云掩膜函数
def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  # Get the pixel QA band.
  qa = image.select('pixel_qa')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

def NDVI(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')  
    return image.addBands(ndvi)

def mNDWI(image):
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    return image.addBands(mndwi)

# 可视化参数设置
vis_params = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

In [7]:
# 筛选数据（使用日期，研究区），去云
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(wuhan_ee) \
    .filterDate('{}-01-01'.format(year), '{}-12-31'.format(year)) \
    .map(maskL8sr) \
    .map(NDVI) \
    .map(mNDWI)

In [ ]:
# # Use these bands for prediction.
# bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# # 分类标签
# label = 'waterclass'

# def image_sample(image):
#     wuhanregion = image.clip(wuhan_ee)
#     wuhan_mNDWI_NDVI_water = wuhanregion.select('mNDWI').gt(wuhanregion.select('NDVI')).eq(1).selfMask()
#     wuhan_mNDWI_NDVI_nowater = wuhanregion.select('mNDWI').gt(wuhanregion.select('NDVI')).eq(0).selfMask()
#     wuhan_water = wuhan_mNDWI_NDVI_water.select('mNDWI').rename('waterclass')
#     wuhan_nowater = wuhan_mNDWI_NDVI_nowater.select('mNDWI').rename('waterclass')
#     points_water = wuhan_water.sample(**{
#         'region': wuhan_ee,
#         'scale': 30,
#         'numPixels': 40,
#         'seed': 0,
#         'geometries': True  # Set this to False to ignore geometries
#     })
#     points_nowater = wuhan_nowater.sample(**{
#         'region': wuhan_ee,
#         'scale': 30,
#         'numPixels': 40,
#         'seed': 0,
#         'geometries': True  # Set this to False to ignore geometries
#     }) 
#     points = points_water.merge(points_nowater)
#     sample_point = image.select(bands).sampleRegions(**{
#         'collection': points,
#         'properties': [label],
#         'scale': 30
#     })
# #     Map.addLayer(points_water.merge(points_nowater),{},'POINT')
#     return sample_point

In [8]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# 分类标签
label = 'waterclass'

def image_sample(image):
    wuhanregion = image.clip(wuhan_ee)
    wuhan_mNDWI_NDVI = wuhanregion.select('mNDWI').gt(wuhanregion.select('NDVI'))
    wuhan_water = wuhan_mNDWI_NDVI.select('mNDWI').rename('waterclass')
    points = wuhan_water.sample(**{
        'region': wuhan_ee,
        'scale': 30,
        'numPixels': 50,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    sample_point = image.select(bands).sampleRegions(**{
        'collection': points,
        'properties': [label],
        'scale': 30
    })
#     Map.addLayer(points_water.merge(points_nowater),{},'POINT')
    return sample_point

In [9]:
sample_points = image_sample(ee.Image(landsat_images.toList(landsat_images.size()).get(0)))
for i in range(1,landsat_images.size().getInfo(),1):
    points_i = image_sample(ee.Image(landsat_images.toList(landsat_images.size()).get(i)))
    sample_points = sample_points.merge(points_i)
# print(sample_points.size().getInfo())
# print(sample_points.first().getInfo())

In [10]:
# 在training要素集中增加一个random属性，值为0到1的随机数
withRandom = sample_points.randomColumn('random')

#把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
split = 0.7
trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
testingPartition = withRandom.filter(ee.Filter.gte('random', split))

In [ ]:
# 利用70%数据随机森林进行训练
trainedClassifier = ee.Classifier.smileRandomForest(20).train(sample_points,label,bands)
#对Landsat-8进行分类
def classification(image):
    return image.select(bands).classify(trainedClassifier)
result_images = landsat_images.map(classification)

In [ ]:
wuhan_water = result_iamges.sum()
Map.addLayer(wuhan_water,{'palette':['white','#99ffb8','#685eff']},'wuhan_water')

In [ ]:
Map